##### Loading packages

In [1]:
from pymongo import MongoClient
import json
import time
from bson.json_util import dumps
from elasticsearch import Elasticsearch
import pandas as pd
from py2neo import Graph, Node, Relationship
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import treetaggerwrapper
import warnings
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
from time import time, ctime
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/home/ubuntu/anaconda3/envs/audal/lib/python3.8/site-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
start = time()

##### Connection to MongoDB

In [3]:
#Server
server = MongoClient('')
#BD
db = server['aura_pmi_dl']
#Collection
collection = db['document_profiles']

##### Connection to Neo4j

In [4]:
graph = Graph("bolt://host:7687", auth=("neo4j", ""))

##### Connection to Elasticsearch

In [5]:
es = Elasticsearch("host")

##### Loading stopwords

In [10]:
#FR
with open('/home/ubuntu/aura-pmi/Ressources/french_stopwords.txt', 'r', encoding="utf8") as f:
    custom_stpw_fr = f.read().split()
nltk_stopwords_fr = stopwords.words("french")
stpw_fr = set(custom_stpw_fr+nltk_stopwords_fr) 

In [12]:
#EN
stpw_en = list(set(stopwords.words('english')))
with open('/home/ubuntu/aura-pmi/Ressources/english_stopwords.txt', 'r', encoding="utf8") as f:
    custom_stpw_en = f.read().split()
nltk_stopwords_en = stopwords.words("english")
stpw_en = set(custom_stpw_en+nltk_stopwords_en) 

##### Lemmatization parameters

In [8]:
tagger_fr = treetaggerwrapper.TreeTagger(TAGLANG='fr')

In [9]:
tagger_en = treetaggerwrapper.TreeTagger(TAGLANG='en')

##### Data in french


In [10]:
##### DATA IN FRENCH


#First query to check number of hits
query = {"query": {"match_all": {}}}
res = es.search(index="document_index_fr", body=query, scroll='2m',)
result_size = res['hits']['total']['value']
print("Got %d Hits:" % result_size)
# Get the scroll ID
first = True
sid = res['_scroll_id']
scroll_size = len(res['hits']['hits'])


while ((scroll_size > 0) | first):
  
    liste_text_fr = []
    liste_id_fr = []
    
    for hit in res['hits']['hits']:
        
        identifier = hit['_source']['identifier']
        text_fr = hit['_source']['content']
        liste_text_fr.append(text_fr)
        liste_id_fr.append(identifier)
    
    liste_lemmatized_text_fr = []
    for text_fr, id_fr in zip(liste_text_fr, liste_id_fr):
        
        #PROCESSING THE RESULT
        
        #Stopwords removal
        word_tokens = word_tokenize(text_fr, language='french') 
        filtered_text = [w for w in word_tokens if w not in stpw_fr] 
        filtered_text =[term.lower() for term in filtered_text if term.isalpha()]
        filtered_text_fr = " ".join(filtered_text)
       
        #Lemmatization
        
        text = filtered_text_fr.lower()
        tags = tagger_fr.tag_text(text)
        #liste_lemmas = [tag.split('\t')[2] for tag in tags]
        liste_lemmas = []
        for tag in tags: 
            tag_splited = tag.split('\t')
            if len(tag_splited) > 2:
                if tag_splited[1] not in ["DET:ART", "PRP", "KON", "PRP:det"]: #First punctuation and articles filtering
                    liste_lemmas.append(tag_splited[2])
        lemmatized_text_fr = " ".join(liste_lemmas)
        liste_lemmatized_text_fr.append(lemmatized_text_fr)    

        
    #VECTORIZATION
    vectorizer = CountVectorizer(max_features=1000)
    mdt = vectorizer.fit_transform(liste_lemmatized_text_fr)
    vocabulary = vectorizer.get_feature_names()
    vectors = mdt.toarray()
    #Insertion in Neo4j and MongoDB
    for i, identifier in enumerate(liste_id_fr):
       
        #Insertion in Neo4J
        graph.run('CREATE (c:Document:Refined {identifier:"'+ identifier +'", vocabulary:"global_vocabulary", format:"key-values"}) RETURN c')
        ##### Linking representation nodes with main documents
        query = '''MATCH (d:Object),(p:Refined {name:"global_vocabulary", format:"key-values"}) WHERE d.identifier = p.identifier
                    CREATE (d)-[r:REPRESENTATION]->(p)
                    RETURN d,p'''
        graph.run(query)
    
        #Insertion in MongoDB
        doc_vector = vectors[i]
        doc_vocab = pd.DataFrame({"term":vocabulary, "freq":doc_vector}) 
        doc_vocab.index = doc_vocab["term"]
        #print(doc_vocab.head())
        doc_vocab = doc_vocab[doc_vocab['freq'] > 0]
        representation = doc_vocab.to_dict(orient='records') #Generation of the representation
        query = {'_id': identifier}
        new_values = {'$addToSet': {'representations': {"vocabulary":"global_vocabulary", "format":"key-values", "data":representation} }}
        collection.update_one(query, new_values)
    
    ##Query again
    res = es.scroll(scroll_id=sid, scroll='2m')
    # Update the scroll ID
    first = False
    sid = res['_scroll_id']
    scroll_size = len(res['hits']['hits'])
    
print("END")
    

Got 8649 Hits:
END


##### Data in english

In [11]:
##### DATA IN ENGLISH


#First query to check number of hits
query = {"query": {"match_all": {}}}
res = es.search(index="document_index_en", body=query, scroll='2m',)
result_size = res['hits']['total']['value']
print("Got %d Hits:" % result_size)
# Get the scroll ID
first = True
sid = res['_scroll_id']
scroll_size = len(res['hits']['hits'])


while ((scroll_size > 0) | first):
  
    liste_text_en = []
    liste_id_en = []
    
    for hit in res['hits']['hits']:
        
        identifier = hit['_source']['identifier']
        text_en = hit['_source']['content']
        liste_text_en.append(text_en)
        liste_id_en.append(identifier)
    
    liste_lemmatized_text_en = []
    for text_en, id_en in zip(liste_text_en, liste_id_en):
        
        #PROCESSING THE RESULT
        
        #Stopwords removal
        word_tokens = word_tokenize(text_fr, language='english') 
        filtered_text = [w for w in word_tokens if w not in stpw_en] 
        filtered_text =[term.lower() for term in filtered_text if term.isalpha()]
        filtered_text_en = " ".join(filtered_text)
       
        #Lemmatization
       
        text = filtered_text_en.lower()
        tags = tagger_en.tag_text(text)
        #liste_lemmas = [tag.split('\t')[2] for tag in tags]
        liste_lemmas = []
        for tag in tags: 
            tag_splited = tag.split('\t')
            if len(tag_splited) > 2:
                if tag_splited[1] not in ["DET:ART", "PRP", "KON", "PRP:det"]: #First punctuation and articles filtering
                    liste_lemmas.append(tag_splited[2])
        lemmatized_text_en = " ".join(liste_lemmas)
        liste_lemmatized_text_en.append(lemmatized_text_en)    

        
    #VECTORIZATION
    vectorizer = CountVectorizer(max_features=1000)
    mdt = vectorizer.fit_transform(liste_lemmatized_text_en)
    vocabulary = vectorizer.get_feature_names()
    vectors = mdt.toarray()
    #Insertion in Neo4j and MongoDB
    for i, identifier in enumerate(liste_id_en):
       
        #Insertion in Neo4J
        graph.run('CREATE (c:Document:Refined {identifier:"'+ identifier +'", vocabulary:"global_vocabulary", format:"key-values"}) RETURN c')
        ##### Linking representation nodes with main documents
        query = '''MATCH (d:Object),(p:Refined {name:"alternative_vocabulary"}) WHERE d.identifier = p.identifier
                    CREATE (d)-[r:REPRESENTATION]->(p)
                    RETURN d,p'''
        graph.run(query)
    
        #Insertion in MongoDB
        doc_vector = vectors[i]
        doc_vocab = pd.DataFrame({"term":vocabulary, "freq":doc_vector}) 
        doc_vocab.index = doc_vocab["term"]
        #print(doc_vocab.head())
        doc_vocab = doc_vocab[doc_vocab['freq'] > 0]
        representation = doc_vocab.to_dict(orient='records') #Generation of the representation
        query = {'_id': identifier}
        new_values = {'$addToSet': {'representations': {"vocabulary":"global_vocabulary", "format":"key-values", "data":representation} }}
        collection.update_one(query, new_values)
    
    ##Query again
    res = es.scroll(scroll_id=sid, scroll='2m')
    # Update the scroll ID
    first = False
    sid = res['_scroll_id']
    scroll_size = len(res['hits']['hits'])
    
print("END")
    

Got 41324 Hits:
END


In [12]:
done = time()
elapsed = done - start
print("*"*20)
print("STARTED : "+ctime(start) )
print("END : "+ctime(done) )
print("TIME ELAPSED:" + str(elapsed/60) + " MINUTES" )

********************
STARTED : Wed Jan 27 14:24:45 2021
END : Wed Jan 27 17:56:31 2021
TIME ELAPSED:211.76080036560694 MINUTES
